In [18]:
import sys
sys.path.append('/code/src/')
from my_functions import train_test_time_series, evaluate_forecast, load_dataframe_from_csv, plot_time_series,save_results_to_file

In [19]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
class TimeSeriesPipeline:
    def __init__(self, path_load):
        self.path_load = path_load
        self.data = None
        self.data_without_outliers = None
        self.models = [
            ('Linear Regression', LinearRegression()),
            ('Random Forest', RandomForestRegressor())
        ]
        self.datasets = []
        self.model_metrics = []
        self.tscv = TimeSeriesSplit(n_splits=5, test_size=6)

    def load_data(self, file_name):
        full_path = self.path_load + file_name
        data = load_dataframe_from_csv(full_path)
        data.index = pd.to_datetime(data.index)
        data.index.freq = 'M'
        return data

    def preprocess_data(self):
        file_name = "processed/df_time_monthly.csv"
        file_name_without_outliers = "processed/df_time_monthly_without_outliers.csv"
        self.data = self.load_data(file_name)
        self.data_without_outliers = self.load_data(file_name_without_outliers)
        X = self.data.drop('cost', axis=1)
        y = self.data['cost']
        X_without_outliers = self.data_without_outliers.drop('cost', axis=1)
        y_without_outliers = self.data_without_outliers['cost']
        self.datasets = [
            {'name': 'Normal', 'train_ratio': 0.8, 'X': X, 'y': y},
            {'name': 'Without Outliers', 'train_ratio': 0.8, 'X': X_without_outliers, 'y': y_without_outliers}
            # Add more datasets if needed
        ]

    def fit_arima(self, train_y, train_X):
        model = pm.auto_arima(train_y, exogenous=train_X, seasonal=True)
        return model

    def fit_sarima(self, train_y, train_X, order, seasonal_order):
        model = SARIMAX(train_y, exog=train_X, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        return model_fit

    def evaluate_model(self, model_name, model, train_X, train_y, test_X, test_y):
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        metrics = evaluate_forecast(test_y, y_pred)
        return {
            'model_name': model_name,
            'model_fit': model,
            'metrics': metrics
        }, y_pred

    def evaluate_arima(self, model_arima_fit, train_y, test_y, test_X_df, dataset_name):
        y_pred_arima = model_arima_fit.predict(n_periods=len(test_y), exogenous=test_X_df.values)
        # y_pred_arima = model_arima_fit.predict(start=len(train_y), end=len(train_y) + len(test_y) - 1,
        #                                        exog=test_X_df.values)
        metrics_arima = evaluate_forecast(test_y, y_pred_arima)
        return {
            'model_name': 'Auto ARIMA',
            'model_fit': model_arima_fit,
            'metrics': metrics_arima
        }, y_pred_arima

    def evaluate_sarima(self, model_sarima_fit, train_y, test_y, test_X_df, dataset_name):
        y_pred_sarima = model_sarima_fit.predict(start=len(train_y), end=len(train_y) + len(test_y) - 1,
                                                 exog=test_X_df.values)
        metrics_sarima = evaluate_forecast(test_y, y_pred_sarima)
        return {
            'model_name': 'SARIMA',
            'model_fit': model_sarima_fit,
            'metrics': metrics_sarima
        }, y_pred_sarima

    def train_and_evaluate(self):
        for dataset in self.datasets:
            dataset_name = dataset['name']
            X = dataset['X'].values
            y = dataset['y'].values
            for train_index, test_index in self.tscv.split(X):
                train_X, test_X = X[train_index], X[test_index]
                train_y, test_y = y[train_index], y[test_index]
                test_X_df = pd.DataFrame(test_X, index=pd.to_datetime(dataset['X'].index[test_index]))

                model_arima_fit = self.fit_arima(train_y, train_X)
                model_sarima_fit = self.fit_sarima(train_y, train_X, model_arima_fit.order, model_arima_fit.seasonal_order)

                for model_name, model in self.models:
                    result, y_pred = self.evaluate_model(model_name, model, train_X, train_y, test_X, test_y)
                    self.model_metrics.append(result)
                    y_pred_series = pd.Series(y_pred, index=test_X_df.index)
                    plot_time_series(pd.Series(train_y, index=pd.to_datetime(dataset['X'].index[train_index])),
                                     pd.Series(test_y, index=pd.to_datetime(dataset['X'].index[test_index])),
                                     y_pred_series,
                                     f'Serie Tiempo - {dataset_name} - {model_name}')
                    print('---')

                result_arima, y_pred_arima = self.evaluate_arima(model_arima_fit, train_y, test_y, test_X_df, dataset_name)
                self.model_metrics.append(result_arima)
                y_pred_arima_series = pd.Series(y_pred_arima, index=test_X_df.index)
                plot_time_series(pd.Series(train_y, index=pd.to_datetime(dataset['X'].index[train_index])),
                                 pd.Series(test_y, index=pd.to_datetime(dataset['X'].index[test_index])),
                                 y_pred_arima_series,
                                 f'Serie Tiempo - {dataset_name} - Auto ARIMA')
                print('---')

                result_sarima, y_pred_sarima = self.evaluate_sarima(model_sarima_fit, train_y, test_y, test_X_df, dataset_name)
                self.model_metrics.append(result_sarima)
                y_pred_sarima_series = pd.Series(y_pred_sarima, index=test_X_df.index)
                plot_time_series(pd.Series(train_y, index=pd.to_datetime(dataset['X'].index[train_index])),
                                 pd.Series(test_y, index=pd.to_datetime(dataset['X'].index[test_index])),
                                 y_pred_sarima_series,
                                 f'Serie Tiempo - {dataset_name} - SARIMA')
                print('---')

    def save_results(self, file_name):
        save_results_to_file(self.model_metrics, file_name)





In [17]:
path_load = "../../data/"
pipeline = TimeSeriesPipeline(path_load)
pipeline.preprocess_data()
pipeline.train_and_evaluate()
pipeline.save_results('../../models/results/model_metrics_summary.csv')

╒══════════════════════════════════════════════════╤════════════════════════╕
│ Metric                                           │ Value                  │
╞══════════════════════════════════════════════════╪════════════════════════╡
│ **Scale-Dependent Metrics**                      │                        │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 2.856.488.096.793,95 │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 1.588.710,20         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 1.690.114,82         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ **Percentage-Error Metrics**                     │                        │
├──────────────────────────────────────────────────┼────────────

---
╒══════════════════════════════════════════════════╤════════════════════════╕
│ Metric                                           │ Value                  │
╞══════════════════════════════════════════════════╪════════════════════════╡
│ **Scale-Dependent Metrics**                      │                        │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 1.584.649.196.507,19 │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 1.095.329,49         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 1.258.828,50         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ **Percentage-Error Metrics**                     │                        │
├──────────────────────────────────────────────────┼────────

---
╒══════════════════════════════════════════════════╤══════════════════════╕
│ Metric                                           │ Value                │
╞══════════════════════════════════════════════════╪══════════════════════╡
│ **Scale-Dependent Metrics**                      │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 385.946.123.551,65 │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 532.357,62         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 621.245,62         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ **Percentage-Error Metrics**                     │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean A

---
╒══════════════════════════════════════════════════╤════════════════════════╕
│ Metric                                           │ Value                  │
╞══════════════════════════════════════════════════╪════════════════════════╡
│ **Scale-Dependent Metrics**                      │                        │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 2.437.273.118.160,71 │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 1.298.591,93         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 1.561.176,84         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ **Percentage-Error Metrics**                     │                        │
├──────────────────────────────────────────────────┼────────

---
╒══════════════════════════════════════════════════╤════════════════════════╕
│ Metric                                           │ Value                  │
╞══════════════════════════════════════════════════╪════════════════════════╡
│ **Scale-Dependent Metrics**                      │                        │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 3.147.973.966.444,61 │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 1.612.697,25         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 1.774.253,07         │
├──────────────────────────────────────────────────┼────────────────────────┤
│ **Percentage-Error Metrics**                     │                        │
├──────────────────────────────────────────────────┼────────

...

---
╒══════════════════════════════════════════════════╤══════════════════════╕
│ Metric                                           │ Value                │
╞══════════════════════════════════════════════════╪══════════════════════╡
│ **Scale-Dependent Metrics**                      │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 232.821.702.574,06 │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 305.496,65         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 482.516,01         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ **Percentage-Error Metrics**                     │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean A

---
╒══════════════════════════════════════════════════╤══════════════════════╕
│ Metric                                           │ Value                │
╞══════════════════════════════════════════════════╪══════════════════════╡
│ **Scale-Dependent Metrics**                      │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 537.158.280.000,31 │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 676.346,36         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 732.910,83         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ **Percentage-Error Metrics**                     │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean A

---
╒══════════════════════════════════════════════════╤══════════════════════╕
│ Metric                                           │ Value                │
╞══════════════════════════════════════════════════╪══════════════════════╡
│ **Scale-Dependent Metrics**                      │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Squared Error (MSE):                        │ $ 162.528.939.620,06 │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean Absolute Error (MAE):                       │ $ 367.040,93         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Root Mean Squared Error (RMSE):                  │ $ 403.148,78         │
├──────────────────────────────────────────────────┼──────────────────────┤
│ **Percentage-Error Metrics**                     │                      │
├──────────────────────────────────────────────────┼──────────────────────┤
│ Mean A

---


NameError: name 'save_results_to_file' is not defined